# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from regression_bias_corrector import LinearBiasCorrector
import optuna
from sklearn.preprocessing import LabelEncoder
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import matplotlib.lines as mlines
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats
from functools import reduce
from scipy import stats
import shutil
import os
from pypdf import PdfReader
import re
import logging
from slugify import slugify

logging.getLogger("pypdf").setLevel(logging.ERROR)


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Load components

In [ ]:
dir_root = f"C:/Work/MillenniumAge"

feat_trgt = 'Возраст'

path_models = 'C:/YandexDisk/Work/bbd/millennium/models/small_with_fixes/Оценка состава тела/M'

model_ids = pd.read_excel(f"{path_models}/models/DANet/selected.xlsx", index_col=0).index.values
df_models = pd.DataFrame(index=model_ids, columns=['sum'])

fn_data = '4'
data = pd.read_excel(f"C:/YandexDisk/Work/bbd/millennium/Результаты чекап/4/{fn_data}.xlsx", index_col=0)

for model_id in model_ids:

    components = {
        # 'Оценка состава тела, женщины': {
        #     'name': 'Оценка состава тела',
        #     'path': f"{dir_root}/data/small_models/Оценка состава тела/F",
        #     'bkg_count': 10,
        #     'likelihood': 0.60
        # },
        'Оценка состава тела, мужчины': {
            'name': 'Оценка состава тела',
            'path': f"{path_models}",
            'path_model': f"{path_models}/models/DANet/{model_id}",
            'bkg_count': 10,
            'likelihood': 0.6
        },
        
        # 'Электрокардиограмма, все': {
        #     'name': 'Электрокардиограмма',
        #     'path': f"{dir_root}/data/small_models/Электрокардиограмма",
        #     'bkg_count': 15,
        #     'likelihood': 0.50
        # },
        
        # 'Гематологические исследования, женщины': {
        #     'name': 'Гематологические исследования',
        #     'path': f"{dir_root}/data/small_models/Гематологические исследования/F",
        #     'bkg_count': 20,
        #     'likelihood': 0.4
        # },
        # 'Гематологические исследования, мужчины': {
        #     'name': 'Гематологические исследования',
        #     'path': f"{dir_root}/data/small_models/Гематологические исследования/M",
        #     'bkg_count': 20,
        #     'likelihood': 0.4
        # },
        
        # 'Биохимические исследования, женщины': {
        #     'name': 'Биохимические исследования',
        #     'path': f"{path_models}",
        #     'path_model': f"{path_models}/models/DANet/{model_id}",
        #     'bkg_count': 12,
        #     'likelihood': 0.90
        # },
        # 'Биохимические исследования, мужчины': {
        #     'name': 'Биохимические исследования',
        #     'path': f"{path_models}",
        #     'path_model': f"{path_models}/models/DANet/{model_id}",
        #     'bkg_count': 12,
        #     'likelihood': 0.90
        # },
        
        # 'Половые гормоны, женщины': {
        #     'name': 'Половые гормоны',
        #     'path': f"{dir_root}/data/small_models/Половые гормоны/F",
        #     'bkg_count': 15,
        #     'likelihood': 0.90
        # },
        # 'Половые гормоны, мужчины': {
        #     'name': 'Половые гормоны',
        #     'path': f"{dir_root}/data/small_models/Половые гормоны/M",
        #     'bkg_count': 12,
        #     'likelihood': 0.90
        # },
        
        # 'Гормоны, все': {
        #     'name': 'Гормоны',
        #     'path': f"{dir_root}/data/small_models/Гормоны",
        #     'bkg_count': 10,
        #     'likelihood': 0.40
        # },
    }
    feats_all = ['Пол']
    feats_pred_all = []
    feats_input_all = []
    feats_aux = []
    for comp in components:
        components[comp]['data'] = pd.read_excel(f"{components[comp]['path']}/data.xlsx", index_col=0)
        components[comp]['feats'] = pd.read_excel(f"{components[comp]['path']}/feats.xlsx", index_col=0)
        components[comp]['results'] = pd.read_excel(f"{components[comp]['path_model']}/df.xlsx", index_col=0)
        components[comp]['metrics'] = pd.read_excel(f"{components[comp]['path_model']}/metrics.xlsx", index_col=0)
        components[comp]['model'] = TabularModel.load_model(f"{components[comp]['path_model']}")
        components[comp]['corrector'] = LinearBiasCorrector()
        comp_results = components[comp]['results']
        components[comp]['corrector'].fit(comp_results.loc[comp_results['Group'] == 'Train', feat_trgt].values, comp_results.loc[comp_results['Group'] == 'Train', 'Prediction'].values)
        res_cols = ['Group', 'Prediction', 'Error', 'Prediction Unbiased', 'Error Unbiased']
        components[comp]['data'].loc[components[comp]['data'].index, res_cols] = comp_results.loc[components[comp]['data'].index, res_cols]
        components[comp]['data_shap'] = components[comp]['data'].copy()
        
        feats = components[comp]['feats'].index.values
        feats = feats[feats != feat_trgt]
        feats_all += list(feats)
        feats_pred_all += [f"Предсказание {components[comp]['name']}", f"Возрастная Акселерация {components[comp]['name']}", f"Модель {components[comp]['name']}",]
        feats_all += [f"Модель {comp}", f"Предсказание {comp}", f"Возрастная Акселерация {comp}"]
        feats_aux += [f"Модель {comp}", f"Предсказание {comp}", f"Возрастная Акселерация {comp}"]
        feats_input_all += list(feats)
        
        components[comp]['feats_corr'] = pd.DataFrame(index=feats, columns=['Correlation'])
        for f in feats:
            components[comp]['feats_corr'].at[f, 'Correlation'], _ = scipy.stats.pearsonr(components[comp]['data'].loc[:, f].values, components[comp]['data'].loc[:, feat_trgt].values)

    feats_all = list(dict.fromkeys(feats_all)) + list(dict.fromkeys(feats_pred_all))
    feats_input_all = list(dict.fromkeys(feats_input_all))
    feats_aux = list(dict.fromkeys(feats_aux))

    for comp in components:
        print(f"{comp}: {components[comp]['data'].shape[0]}")



    # Predict age acceleration for target models
    data_models = pd.DataFrame(columns=feats_all)
    data_models_imputed = pd.DataFrame(columns=feats_input_all)

    df_correspondence = pd.DataFrame()

    for sample_id in tqdm(data.index.values, desc='Вычисление биологического возраста'):
        
        data_models.at[sample_id, "Пол"] = data.at[sample_id, "Пол"]
        
        df_correspondence.at[sample_id, "Пол"] = data.at[sample_id, "Пол"]
        
        components_trgt = []
            
        if data.at[sample_id, "Пол"] == 'Ж':
             
            comps_f = [
                'Оценка состава тела, женщины',
                'Половые гормоны, женщины',
                'Гематологические исследования, женщины',
                'Биохимические исследования, женщины',
                'Электрокардиограмма, все',
                'Гормоны, все' 
            ]
            
            for comp in comps_f:
                if comp in components:
                    components_trgt.append(comp)
            
        elif data.at[sample_id, "Пол"] == 'М':
            
            comps_m = [
                'Оценка состава тела, мужчины',
                'Половые гормоны, мужчины',
                'Гематологические исследования, мужчины',
                'Биохимические исследования, мужчины',
                'Электрокардиограмма, все',
                'Гормоны, все' 
            ]
            
            for comp in comps_m:
                if comp in components:
                    components_trgt.append(comp)
        
        nan_part = 0.1
        
        n_pos = 0
        n_neg = 0
        comp_present = []
        for comp in components_trgt:
            data_models.at[sample_id, f"Модель {comp}"] = False
            feats = components[comp]['feats'].index.values
            feats_w_trgt = list(feats) + [feat_trgt]
            n_feats = len(feats)
            if set(feats).issubset(data.columns):
                data_models_imputed.loc[sample_id, feats] = data.loc[sample_id, feats].isna()
                n_nans = data.loc[sample_id, feats].isna().sum()
                # rho = components[comp]['metrics'].at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['likelihood']
                # rho = components[comp]['metrics'].at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['feats_corr']['Correlation'].abs().max()
                rho = components[comp]['metrics'].at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['feats_corr']['Correlation'].abs().max() * components[comp]['likelihood']
                if n_nans / n_feats < nan_part:
                    comp_present.append(comp)
                    data_sample = data.loc[[sample_id], feats_w_trgt]
                    pred = components[comp]['model'].predict(data_sample)[f'{feat_trgt}_prediction'].values
                    pred = components[comp]['corrector'].predict(pred)
                    data_sample.at[sample_id, f"Предсказание {components[comp]['name']}"] = pred
                    data_sample.at[sample_id, f"Предсказание {comp}"] = pred
                    
                    gt = data_sample.at[sample_id, feat_trgt]
                    aa = pred - gt
                    if aa > 0:
                        n_pos += 1
                    else:
                        n_neg += 1
                    
                    data_sample.at[sample_id, f"Возрастная Акселерация {components[comp]['name']}"] = aa * rho
                    data_sample.at[sample_id, f"Возрастная Акселерация {comp}"] = aa * rho
                    
                    data_sample.at[sample_id, f"Модель {components[comp]['name']}"] = comp
                    
                    data_models.loc[sample_id, data_sample.columns] = data_sample.loc[sample_id, data_sample.columns]
                    
                    # Explainability checking
                    
                    trgt_age = data_models.at[sample_id, feat_trgt]
                    
                    feats = components[comp]['feats'].index.values
                    feats_corr = components[comp]['feats_corr']
                    metrics = components[comp]['metrics']
                    data_shap = components[comp]['data_shap']
                    
                    def predict_func(X):
                        X_df = pd.DataFrame(data=X, columns=feats)
                        y = components[comp]['model'].predict(X_df)[f'{feat_trgt}_prediction'].values
                        y = components[comp]['corrector'].predict(y)
                        return y
                    
                    mae = metrics.at['Test', 'mean_absolute_error_unbiased']
                    # rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['likelihood']
                    # rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * feats_corr['Correlation'].abs().max()
                    rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * feats_corr['Correlation'].abs().max() * components[comp]['likelihood']
                    
                    components[comp]['data']['Биологический возраст'] = components[comp]['data'][feat_trgt] + rho * components[comp]['data']['Error Unbiased']
                    components[comp]['data']['Акселерация'] = rho * components[comp]['data']['Error Unbiased']
                    
                    bkg_count = components[comp]['bkg_count']
                
                    trgt_pred_raw = data_models.at[sample_id, f"Предсказание {comp}"]
                    
                    data_closest = data_shap.loc[data_shap['Error Unbiased'].abs() < mae * rho, :]
                    data_closest = data_closest.iloc[(data_closest['Prediction Unbiased'] - trgt_age).abs().argsort()[:bkg_count]]
                    
                    print(f"Background count: {data_closest.shape[0]}")
                    print(f"Background min diff: {(data_closest['Prediction Unbiased'] - trgt_age).min()}")
                    print(f"Background max diff: {(data_closest['Prediction Unbiased'] - trgt_age).max()}")
                    
                    explainer = shap.SamplingExplainer(predict_func, data_closest.loc[:, feats].values)
                    shap_values = explainer.shap_values(data_models.loc[[sample_id], feats].values)[0]
                    shap_values = shap_values * (trgt_pred_raw - trgt_age) / (trgt_pred_raw - explainer.expected_value)
                    print(f"SHAP values difference: {sum(shap_values) - (trgt_pred_raw - trgt_age)}")
                    # shap_values = shap_values * rho / max(n_pos - 1, n_neg - 1, 1)
                    shap_values = shap_values * rho
                    
                    # SHAP values correction 1
                    shap_corr_thld = 3.0
                    shap_corr_to = 1.0
                    shap_mean_abs = np.mean(np.abs(shap_values))
                    if shap_mean_abs > shap_corr_thld and abs(sum(shap_values)) < shap_corr_thld:
                        print('SHAP values correction 1')
                        shap_pos_ids = np.argwhere(shap_values >= 0).ravel()
                        shap_neg_ids = np.argwhere(shap_values < 0).ravel()
                        
                        shap_pos_sum_abs = np.sum(np.abs(shap_values[shap_pos_ids]))
                        shap_neg_sum_abs = np.sum(np.abs(shap_values[shap_neg_ids]))
                        
                        shap_sum_abs_from = np.sum(np.abs(shap_values))
                        shap_sum_abs_to = shap_corr_to * len(shap_values)
                        
                        shap_corr_diff = shap_sum_abs_from - shap_sum_abs_to
                        
                        for pos_id in shap_pos_ids:
                            curr_part = abs(shap_values[pos_id]) / shap_pos_sum_abs
                            shap_values[pos_id] -= curr_part * shap_corr_diff * 0.5
                        for neg_id in shap_neg_ids:
                            curr_part = abs(shap_values[neg_id]) / shap_neg_sum_abs
                            shap_values[neg_id] += curr_part * shap_corr_diff * 0.5
                            
                    # SHAP values correction 2
                    shap_corr_max_thld = 1.0
                    shap_corr_max_to = 0.95
                    shap_max_abs = np.max(np.abs(shap_values))
                    shap_abs_sum = np.abs(np.sum(shap_values))
                    if shap_max_abs > shap_corr_max_thld * shap_abs_sum:
                        print('SHAP values correction 2')
                        shap_pos_ids = np.argwhere(shap_values >= 0).ravel()
                        shap_neg_ids = np.argwhere(shap_values < 0).ravel()
                        
                        shap_pos_sum_abs = np.sum(np.abs(shap_values[shap_pos_ids]))
                        shap_neg_sum_abs = np.sum(np.abs(shap_values[shap_neg_ids]))
                        
                        shap_corr_diff = (shap_max_abs - shap_corr_max_thld) * shap_corr_max_to
                        
                        for pos_id in shap_pos_ids:
                            curr_part = abs(shap_values[pos_id]) / shap_pos_sum_abs
                            shap_values[pos_id] -= curr_part * shap_corr_diff
                        for neg_id in shap_neg_ids:
                            curr_part = abs(shap_values[neg_id]) / shap_neg_sum_abs
                            shap_values[neg_id] += curr_part * shap_corr_diff
                    
                    df_comp = pd.DataFrame(index=feats, columns=['SHAP', 'Values', 'Correlation', 'Percentile', 'Class', 'Consistent', 'Show'])
                    df_comp['SHAP'] = shap_values
                    df_comp['Imputed?'] = False
                    df_comp.sort_values(by='SHAP', key=abs, inplace=True)
                    df_comp.loc[df_comp.index.values, 'Values'] = data_models.loc[sample_id, df_comp.index.values].values
                    df_comp.loc[df_comp.index.values, 'Correlation'] = feats_corr.loc[df_comp.index.values, 'Correlation']
                    for f_id, f in enumerate(df_comp.index.values):
                        df_comp.at[f, 'Percentile'] = scipy.stats.percentileofscore(data_closest.loc[:, f].values, data_models.at[sample_id, f])
                        if (
                            ((df_comp.at[f, 'Correlation'] > 0) & (df_comp.at[f, 'SHAP'] > 0) & (df_comp.at[f, 'Percentile'] > 60)) or \
                            ((df_comp.at[f, 'Correlation'] > 0) & (df_comp.at[f, 'SHAP'] < 0) & (df_comp.at[f, 'Percentile'] < 40)) or \
                            ((df_comp.at[f, 'Correlation'] < 0) & (df_comp.at[f, 'SHAP'] > 0) & (df_comp.at[f, 'Percentile'] < 40)) or \
                            ((df_comp.at[f, 'Correlation'] < 0) & (df_comp.at[f, 'SHAP'] < 0) & (df_comp.at[f, 'Percentile'] > 60))
                            ):
                            df_comp.at[f, 'Consistent'] = 1
                        else:
                            df_comp.at[f, 'Consistent'] = 0
                        # If SHAP values too small
                        if abs(df_comp.at[f, 'SHAP']) < 0.1:
                            df_comp.at[f, 'Show'] = 0
                        df_correspondence.at[sample_id, f"{f} consistent"] = df_comp.at[f, 'Consistent']
                        df_correspondence.at[sample_id, f"{f} order"] = f_id
                        df_correspondence.at[sample_id, f"{f} Percentile"] = df_comp.at[f, 'Percentile']
                        df_correspondence.at[sample_id, f"{f} SHAP"] = df_comp.at[f, 'SHAP']
                        df_correspondence.at[sample_id, f"{f} Value"] = df_comp.at[f, 'Values']
                        df_correspondence.at[sample_id, f"{f} Correlation"] = df_comp.at[f, 'Correlation']
                
        if len(comp_present) > 0:
            data_models.at[sample_id, "Число моделей"] = len(comp_present)
            data_models.at[sample_id, "Число моделей c отрицательной аккселерацией"] = n_neg
            data_models.at[sample_id, "Число моделей c положительной аккселерацией"] = n_pos
            data_models.at[sample_id, "Возрастная Акселерация"] = 0.0
            for comp in comp_present:
                data_models.at[sample_id, f"Модель {comp}"] = True
                # samples.at[sample_id, f"Возрастная Акселерация {components[comp]['name']}"] /= max(n_pos - 1, n_neg - 1, 1)
                # samples.at[sample_id, f"Возрастная Акселерация {comp}"] /= max(n_pos - 1, n_neg - 1, 1)
                data_models.at[sample_id, "Возрастная Акселерация"] += data_models.at[sample_id, f"Возрастная Акселерация {comp}"]

    for comp in components:       
        total_score = 0
        feats = components[comp]['feats'].index.values
        for f in feats:
            total_score += df_correspondence.loc[df_correspondence[f"{f} order"] > len(feats) - 2 - 1, f"{f} consistent"].sum()

    
    df_models.at[model_id, 'sum'] = total_score
    
    df_correspondence.to_excel(f"{path_models}/models/DANet/{model_id}/correspondence_{fn_data}.xlsx")

df_models.to_excel(f"{path_models}/models/DANet/correspondence.xlsx")